In [9]:
import os,sys,shutil,glob,argparse
import numpy as np
from PIL import Image
from lxml import etree
import cv2

In [19]:
voc="./VOC_OxfordHandDataset/"
fashion_dataset_path = './annotation'

In [11]:
def make_directories():
    if os.path.exists(voc):
        prompt = raw_input('directory already exist. overwrite? (yes,no):')
        if prompt == 'no':
            exit(0)
        shutil.rmtree(voc)
    os.makedirs(voc)
    os.makedirs(voc+"Annotations")
    os.makedirs(voc+"ImageSets")
    os.makedirs(voc+"ImageSet/Layout")
    os.makedirs(voc+"JPEGImages")

In [29]:
def write_voc_file(fname,label,coord,img_width,img_height):
    annotation = etree.Element('annotation')
    filename = etree.Element('filename')
    filename.text = fname
    annotation.append(filename)
    folder = etree.Element('folder')
    folder.text = "Hands"
    annotation.append(folder)
    object = etree.Element('object')
    annotation.append(object)
    name = etree.Element('name')
    name.text = label
    object.append(name)
    bndbox = etree.Element('bndbox')
    object.append(bndbox)
    xmax = etree.Element('xmax')
    xmax.text = str(coord[2])
    bndbox.append(xmax)
    xmin = etree.Element('xmin')
    xmin.text = str(coord[0])
    bndbox.append(xmin)
    ymax = etree.Element('ymax')
    ymax.text = str(coord[3])
    bndbox.append(ymax)
    ymin = etree.Element('ymin')
    ymin.text = str(coord[1])
    bndbox.append(ymin)
    difficult = etree.Element('difficult')
    difficult.text = '0'
    object.append(difficult)
    occluded = etree.Element('occluded')
    occluded.text = '0'
    object.append(occluded)
    pose = etree.Element('pose')
    pose.text = 'Unspecified'
    object.append(pose)
    truncated = etree.Element('truncated')
    truncated.text = '1'
    object.append(truncated)
    img_size = etree.Element('size')
    annotation.append(img_size)
    depth = etree.Element('depth')
    depth.text = '3'
    img_size.append(depth)
    height = etree.Element('height')
    height.text = str(img_height)
    img_size.append(height)
    width = etree.Element('width')
    width.text = str(img_width)
    img_size.append(width)

    return annotation
    

In [30]:
# ###TEST Function
# fname="one_hand001"
# label="top"
# coord=[10,20,30,40]
# img_width=100
# img_height=300
# annotation = write_voc_file(fname,label ,coord , img_width,img_height)
# et = etree.ElementTree(annotation)
# et.write(voc + "Annotations/" + fname + ".xml", pretty_print=True)

In [31]:
def get_Category_dict():
    with open(fashion_dataset_path ) as category_file:
        next(category_file)
        next(category_file)
        category_dict=dict()
        index=0
        for line in category_file:
            index+=1
            line=line.split()
            category_dict[index]=dict()
            category_dict[index]['item']=line[0]
            typ=int(line[1])
            if typ==1:
                category_dict[index]['type']="Upper_Body"
            elif typ==2:
                category_dict[index]['type']="Lower_Body"
            elif typ==3:
                category_dict[index]['type']="Full_Body"
            else:
                category_dict[index]['type']="Undefined"
    return category_dict        


In [38]:
import pickle
def save_Category_img():
    if os.path.isfile(fashion_dataset_path):
        return
    with open(fashion_dataset_path +".txt") as category_file:
        next(category_file)
        next(category_file)
        category_dict=dict()
        index=0
        for line in category_file:
            line=line.split()
            category_dict[line[0]]=int(line[1])
    pickle.dump(category_dict,open(fashion_dataset_path,'wb'))
    


In [39]:
def generate_dataset():
    count=0
    make_directories()
    save_Category_img()
    
    category_dict=get_Category_dict()
    with open(fashion_dataset_path ) as list_bbox:
        next(list_bbox)
        next(list_bbox)
        imgCategory=pickle.load(open(fashion_dataset_path,'rb'))
        cnt=0
        for line in list_bbox:
                line = line[:-1].split()
                line=" ".join(line)
                line=line.split(' ')
                image_path_name = line[0]
                image_name = line[0].split('/')[-1]
                fname = line[0].replace('/', '_')
                coord=[]
                coord.append(int(line[1]))
                coord.append(int(line[2]))
                coord.append(int(line[3]))
                coord.append(int(line[4]))
                path=fashion_dataset_path+"Img/"+image_path_name
                if os.path.isfile(path)==False:
                    continue
                img = cv2.imread(path)
                img_height, img_width = img.shape[:2]
                shutil.copy(path,voc+"JPEGImages/"+fname)
                label=category_dict[imgCategory[image_path_name]]['type']
                annotation = write_voc_file(fname,label ,coord , img_width,img_height)
                et = etree.ElementTree(annotation)
                et.write(voc + "Annotations/" + fname + ".xml", pretty_print=True)                    
                cnt+=1
                if(cnt%10==0):
                    print(str(cnt)+" /390 files completed...")
        print "Total count="+str(cnt)
generate_dataset()




directory already exist. overwrite? (yes,no):yes


IndexError: list index out of range